In [2]:
'''
All rights can reffer to the LICENSE.md.

Created on July 19, 2018.

This module provides the several classes of autoencoder series.
The use could simply use these API without defining the structure by oneself.

@author: steven.cy.chuang
'''

'\nAll rights can reffer to the LICENSE.md.\n\nCreated on July 19, 2018.\n\nThis module provides the several classes of autoencoder series.\nThe use could simply use these API without defining the structure by oneself.\n\n@author: steven.cy.chuang\n'

In [3]:
import os
import pickle
import json
from time import time
import keras
from keras.layers import Input, Dense, Lambda, Conv2D, Conv2DTranspose, Activation, Flatten, Reshape, LeakyReLU, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, model_from_json
from keras import backend as K
from keras.metrics import *

# Variational autoencoder

In [13]:
class VAE():
    stdEps = 1.0
    
    def __init__(self, 
                 dimInput, 
                 layerDense=[64, 2], actDense='leaky_relu', dropoutDense=0.5,
                 ratRecon=0.998
                ):
        '''
        The basic properties and pipeline will be defined in the initialization.
        It should be noted that layerDense defines the first half(encoder) of network. 
        The decoder will be reflected structure.
        For example, [64, 16, 2] means the nodes of decoder will be [2, 16, 64]. 
        There is another parameter should noted that ratRecon=0.5 doesn't mean the effect is half.
        Because KL loss and reconstruction loss are not the same scale.
        Args:
            dimInput (int): the number of input dimension. All features are flatten as a vector.
            layerDense (list[int]): the numbers of each dense layer. Default is [64, 2].
            actDense (string): the activation function. Default is 'leaky_relu'.
            dropoutDense (float): the dropout layer. Default is 0.5.
            ratRecon (float): the parameter for tuning the effects between KL loss and reconstruction loss.
        '''
        
        # Initialize some setting 
        self.dimInput = dimInput # dimInput is width*height
        self.inputs = Input(shape=(dimInput,)) 
        self.dimEncode = layerDense[-1]
        self.ratRecon = ratRecon
        
        self.encoding(layerDense, actDense, dropoutDense)
        
        self.decoding(layerDense, actDense, dropoutDense)
        
        self.autoencoder = Model(self.inputs, self.decoder(self.encoder(self.inputs)), name='autoencoder')

        
    def encoding(self, layerDense, actDense, dropoutDense):
        dimEncode = self.dimEncode
        x = self.inputs

        # Stack of Dense layers
        for numFilt in layerDense[:-1]:
            if actDense == 'leaky_relu':
                x = Dense(numFilt)(x)
                x = LeakyReLU()(x)
            else:
                x = Dense(numFilt, activation=actDense)(x)
            if dropoutDense > 0:
                x = Dropout(0.5)(x)
        self.zMean = Dense(self.dimEncode)(x)
        self.zSigmaLog = Dense(self.dimEncode)(x) # log for linear dense

        # Define the sampling function for the sampling layer.
        # Note that the function must be in the same location with encoding for saving/loadind model.
        def sampling(args, stdEps):
            zMean, zSigmaLog = args
            epsilon = K.random_normal(shape=(K.shape(zMean)[0], K.shape(zMean)[1]),
                                      mean=0., stddev=stdEps)
            return zMean + K.exp(zSigmaLog) * epsilon  
        
        # Construct the latent as the output and build the encorder pipeline
        z = Lambda(sampling, arguments={'stdEps':self.stdEps})([self.zMean, self.zSigmaLog])
        self.encoder = Model(self.inputs, z, name='encoder')
        
        
    def decoding(self, layerDense, actDense, dropoutDense):
         # Build the Decoder Model
        inputLatent = Input(shape=(self.dimEncode,), name='decoder_input')
        x = inputLatent
        for numFilt in layerDense[-2::-1]:
            if actDense == 'leaky_relu':
                x = Dense(numFilt)(x)
                x = LeakyReLU()(x)
            else:
                x = Dense(numFilt, activation=actDense)(x)
            if dropoutDense > 0:
                x = Dropout(0.5)(x)
            
        # Reconstruct the pixels as the output and build the decorder pipeline
        outputs = Dense(self.dimInput, activation='sigmoid', name='decoder_output')(x)
        self.decoder = Model(inputLatent, outputs, name='decoder')
        
        
    def lossVAE(self, tensorInput, tensorDecode):
        zMean = self.zMean
        zSigmaLog = self.zSigmaLog
        ratRecon = self.ratRecon
        
        lossRecon =  binary_crossentropy(K.flatten(tensorInput), K.flatten(tensorDecode))
#         lossRecon =  metrics.mean_squared_error(K.flatten(tensorInput), K.flatten(tensorDecode))
        lossKL = - 0.5 * K.sum(1 + 2 * zSigmaLog - K.square(zMean) - K.square(K.exp(zSigmaLog)), axis=-1)
        return ratRecon * lossRecon + (1 - ratRecon) * lossKL
        
        
    def fit(self,
            xTrain, xValid,
            numEpochs=50, sizeBatch=32, nameOptim='adam', metrics=None,
            tempPathBest=None, patience=3,
           ):
        '''
        The method is for training process. 
        The users can call this method easily just putting training and validation datasets.
        The dimension of dataset is determined by [#instance, *dimInput].
        For example, dimInput is flatten as a number and the dimension of dataset is [#instance, #feature]
        If dimInput is a list to represent [width, height, channels], the dimension of dataset is [#instance, width, height, channels]
        Args:
            xTrain (numpy ndarray): the training dataset.
            xValid (numpy ndarray): the validation dataset.
            numEpochs (int): the maximal epochs for training. Default is 50.
            sizeBatch (int): the batch size. Default is 32.
            nameOptim (string): the method for optimization. Default is adam.
            metrics (list(string or keras metrics)): the usage is the same with keras metrics for compile 
            tempPathBest (string): the temperory path of the best model for early-stop. Default None means without early-stop. 
            patience (int): the times of epochs to allow further trying if current loss is not better than the best. 
        '''
        self.autoencoder.compile(optimizer=nameOptim, loss=self.lossVAE, metrics=metrics)

        if tempPathBest is None:
            callbacks = None
        else:
            cbEarlyStop = EarlyStopping(monitor='val_loss', patience=patience, verbose=1, mode='auto')
            chkpt = tempPathBest + 'AutoEncoder.{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5'
            cbCheckPoint = ModelCheckpoint(filepath = chkpt, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
            callbacks = [cbEarlyStop, cbCheckPoint]
        
        
        # Train the autoencoder
        tic = time()
        history = self.autoencoder.fit(xTrain, xTrain,
                                       epochs=numEpochs,
                                       batch_size=sizeBatch, shuffle=True,
                                       callbacks=callbacks,
                                       validation_data=(xValid, xValid)
                                      )
        timeTrain = time() - tic
        
        return history, timeTrain
    
    def save(self, pathFolder):
        '''
        Deprecated! Because the availability for saving/loading model is limited.
        The method is to save the models of autoencoders as several hdf5 files in a given path of the folder.
        Args:
            pathFile (string): the given path of the folder where contains encoder, decoder, and autoencoder
        '''
        # Create the message for saving model
        msg = ''
        if os.path.exists(pathFolder):
            msg += 'There is a existing folder.\r\n'
        else:
            os.makedirs(pathFolder)
            msg += 'Create a new folder.\r\n'
        
        # Save the models of encoder and decoder with save()
        self.encoder.save(pathFolder+'/encoder.h5')
        self.decoder.save(pathFolder+'/decoder.h5')
        
        # Save the model of autoencoder with json and save_weights(). 
        # Because autoencoder contains special loss function and sample function.
        self.autoencoder.save_weights(pathFolder+'/weightAutoencoder.h5')
        with open(pathFolder+'/configAutoencoder.json', 'w') as jsonFile:
            jsonFile.write(self.autoencoder.to_json())
        msg += 'successful.\r\n'
        return msg
    
    @staticmethod
    def load(pathFolder):
        '''
        Deprecated! Because the availability for saving/loading model is limited.
        '''
        encoder = keras.models.load_model(pathFolder+'/encoder.h5')
        decoder = keras.models.load_model(pathFolder+'/decoder.h5')
        
        with open(pathFolder+'/configAutoencoder.json', 'r') as jsonFile:                              
            jsonConfig = jsonFile.readlines()[0]
        autoencoder = model_from_json(jsonConfig)
        autoencoder.load_weights(pathFolder+'/weightAutoencoder.h5')
        return encoder, decoder, autoencoder

# Convolutinal VAE

In [14]:
class ConvVAE(VAE):
    
    def __init__(self, 
                 dimInput, 
                 layerConv=[8, 32], sizeKernel=3, strides=2, actConv='relu', padding='same',
                 layerDense=[64, 2], actDense='leaky_relu', dropoutDense=0.5,
                 ratRecon=0.998):
        '''
        The basic properties and pipeline will be defined in the initialization.
        The dimension of input should be a form of a picture presented by a list [width, height, channels].
        It should be noted that layerDense defines the first half(encoder) of network. 
        The decoder will be reflected structure.
        For example, [64, 16, 2] means the nodes of decoder will be [2, 16, 64].
        It is similar for layerConv but decoder is not purely symmetric for convolution layers for this version.
        There is another parameter should noted that ratRecon=0.5 doesn't mean the effect is half.
        Because KL loss and reconstruction loss are not the same scale.
        Args:
            dimInput (list[int]): the dimension of input. E.g. [32, 28, 3] means 32 by 28 RGB pixels.
            layerConv (list[int]): the numbers of each convolution layer. Default is [8, 32].
            sizeKernel (int): the size of filter kernel. Default 3 means 3 by 3.
            strides (int): the stride for convolution. Default is 2.
            actConv (string): the activation function of each convolution layer. Default is 'relu'.
            padding (string): the padding method for convolution. Default is 'same'.
            layerDense (list[int]): the numbers of each dense layer. Default is [64, 2].
            actDense (string): the activation function of each dense layer. Default is 'leaky_relu'.
            dropoutDense (float): the dropout layer. Default is 0.5.
            ratRecon (float): the parameter for tuning the effects between KL loss and reconstruction loss.
        '''
        
        # Initialize some setting 
        self.dimInput = dimInput # dimInput is (width, height, channels)
        self.inputs = Input(shape=(dimInput)) 
        self.dimEncode = layerDense[-1]
        self.ratRecon = ratRecon
        
        self.encoding(layerConv, sizeKernel, strides, actConv, padding,
                      layerDense, actDense, dropoutDense
                     )
        
        self.decoding(layerConv, sizeKernel, strides, actConv, padding,
                      layerDense, actDense, dropoutDense
                     )
        
        self.autoencoder = Model(self.inputs, self.decoder(self.encoder(self.inputs)), name='autoencoder')
        

        
    def encoding(self, 
                 layerConv, sizeKernel, strides, actConv, padding,
                 layerDense, actDense, dropoutDense
                ):
        dimEncode = self.dimEncode
        x = self.inputs
        # Stack of Conv2D layers
        for filters in layerConv:
            x = Conv2D(filters=filters,
                       kernel_size=sizeKernel,
                       strides=strides,
                       activation=actConv,
                       padding=padding)(x)

        # Shape info needed to build Decoder Model
        self.shape = K.int_shape(x)

        # Stack of Dense layers
        x = Flatten()(x)
        for numFilt in layerDense[:-1]:
            if actDense == 'leaky_relu':
                x = Dense(numFilt)(x)
                x = LeakyReLU()(x)
            else:
                x = Dense(numFilt, activation=actDense)(x)
            if dropoutDense > 0:
                x = Dropout(0.5)(x)
        self.zMean = Dense(dimEncode)(x)
        self.zSigmaLog = Dense(dimEncode)(x) # log for linear dense

        # Define the sampling function for the sampling layer.
        # Note that the function must be in the same location with encoding for saving/loadind model.
        def sampling(args, stdEps):
            zMean, zSigmaLog = args
            epsilon = K.random_normal(shape=(K.shape(zMean)[0], K.shape(zMean)[1]),
                                      mean=0., stddev=stdEps)
            return zMean + K.exp(zSigmaLog) * epsilon  
        
        # Construct the latent as the output and build the encorder pipeline
        z = Lambda(sampling, arguments={'stdEps':self.stdEps})([self.zMean, self.zSigmaLog])
        self.encoder = Model(self.inputs, z, name='encoder')

        
    def decoding(self,
                 layerConv, sizeKernel, strides, actConv, padding,
                 layerDense, actDense, dropoutDense
                ):
        
        shape = self.shape
         # Build the Decoder Model
        inputLatent = Input(shape=(self.dimEncode,), name='decoder_input')
        x = inputLatent
        for numFilt in layerDense[-2::-1]:
            if actDense == 'leaky_relu':
                x = Dense(numFilt)(x)
                x = LeakyReLU()(x)
            else:
                x = Dense(numFilt, activation=actDense)(x)
        if dropoutDense > 0:
                x = Dropout(0.5)(x)
        
        x = Dense(shape[1] * shape[2] * shape[3])(x)
        x = Reshape((shape[1], shape[2], shape[3]))(x)

        # Stack of Transposed Conv2D layers
        for numFilt in layerConv[::-1]:
            x = Conv2DTranspose(filters=numFilt,
                                kernel_size=sizeKernel,
                                strides=strides,
                                activation=actConv,
                                padding=padding)(x)

        # Build the Conv2DTranspose layer for the pixel dimension
        x = Conv2DTranspose(filters=self.dimInput[-1],
                            kernel_size=sizeKernel,
#                             strides=strides,
                            padding=padding)(x)

        # Reconstruct the pixels as the output and build the decorder pipeline
        outputs = Activation('sigmoid', name='decoder_output')(x)
        self.decoder = Model(inputLatent, outputs, name='decoder')

# For fast testing

In [15]:
# from time import time
# import unittest
# import numpy as np
# import pickle
# import keras
# from sklearn.metrics import silhouette_score

# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = '1' 

# from keras.datasets import mnist
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# # config.gpu_options.per_process_gpu_memory_fraction = 0.48
# set_session(tf.Session(config=config))
# from keras.datasets import mnist
# import numpy as np
# (xTrain, yTrain), (xTest, yTest) = mnist.load_data()
# xTrain = xTrain.astype('float32') / 255.
# xTest = xTest.astype('float32') / 255.
# numTrain = len(xTrain)
# numTest = len(xTest)
# sizeDigit = xTrain.shape[1:]

# dimInput = np.prod(xTrain.shape[1:])
# xTrain = xTrain.reshape((numTrain, dimInput))
# xTest = xTest.reshape((numTest, dimInput))
# print(xTrain.shape)
# print(xTest.shape)

# numEpochs = 2
# sizeBatch = 128
# sizeKernel = 3
# layerDense = [16, 2]
# ratRecon = 1
# nameOptim = 'adam'
# modelPath = '../model/temp/'
# patience = 3
# stdEps = 1.0

# vae = VAE(dimInput, layerDense=layerDense, ratRecon=ratRecon)
# history, timeTrain = vae.fit(xTrain, xTest, 
#                              numEpochs=numEpochs,
#                              sizeBatch=sizeBatch,
#                              tempPathBest=modelPath)

(60000, 784)
(10000, 784)
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 3s 49us/step - loss: 0.2928 - val_loss: 0.2536

Epoch 00001: val_loss improved from inf to 0.25365, saving model to ../model/temp/AutoEncoder.01-0.29-0.25.hdf5
Epoch 2/2
60000/60000 [==============================] - 2s 38us/step - loss: 0.2513 - val_loss: 0.2464

Epoch 00002: val_loss improved from 0.25365 to 0.24640, saving model to ../model/temp/AutoEncoder.02-0.25-0.25.hdf5
